In [ ]:
from google.colab import drive
import os


drive.mount('/content/gdrive')

root_dir = '/content/gdrive/MyDrive/Teeth X-ray dataset'
assert os.path.isdir(root_dir), f'{root_dir} does not exist'

assert os.path.isdir(root_dir), f'{root_dir} does not exist'

assert len(os.listdir(root_dir)) > 0, f'{root_dir} does not contain any files'


print(f'Total number of images: {len(os.listdir(root_dir))}')


Mounted at /content/gdrive
Total number of images: 7


In [ ]:
import torch
import torchvision.transforms as transforms

class TeethXrayDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_list = os.listdir(root_dir)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        # Get the filename of the image
        filename = self.file_list[index]

        # Load the image using PIL
        image = Image.open(os.path.join(self.root_dir, filename))

        # Apply the transformation if provided
        if self.transform is not None:
            image = self.transform(image)

        # can add additional processing here if needed

        return image, label


# Define the transform for training data
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the image to 256x256 pixels
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the image
])

# Create the dataset and split into train and test sets
dataset = TeethXrayDataset(root_dir='/content/gdrive/MyDrive/Teeth X-ray dataset', transform=train_transform)
train_size = int(0.8 * len(dataset))

# Split the dataset into train and test
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])



In [ ]:
import os
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch


# Define the age classes
age_classes = {
    '0-5': 0,
    '6-12': 1,
    '13-19': 2,
    '20-29': 3,
    '30-59': 4,
    '>60': 5
}

# Define the transforms for training and testing
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


class TeethXrayDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_list = os.listdir(root_dir)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        # Get the filename of the image
        filename = self.file_list[index]

        # Get the age range from the filename
        age_range = get_age_from_filename(filename)

        # Load the image using PIL
        image = Image.open(os.path.join(self.root_dir, filename))

        # Apply the transformation if provided
        if self.transform is not None:
            image = self.transform(image)

        # Map the age range to the corresponding class index
        if age_range == '0-5':
            age_class = 0
        elif age_range == '6-12':
            age_class = 1
        elif age_range == '13-19':
            age_class = 2
        elif age_range == '20-29':
            age_class = 3
        elif age_range == '30-59':
            age_class = 4
        else:
            age_class = 5

        # Convert the image and age class to PyTorch tensors
        image = torch.Tensor(image)
        age_class = torch.tensor(age_class, dtype=torch.long)

        return image, age_class


def get_age_from_filename(filename):
    # Extract the age range from the filename
    age_range = filename.split('_')[1]

    # Map the age range to the corresponding class
    if age_range == '0-5':
        return '0-5'
    elif age_range == '6-12':
        return '6-12'
    elif age_range == '13-19':
        return '13-19'
    elif age_range == '20-29':
        return '20-29'
    elif age_range == '30-59':
        return '30-59'
    else:
        return '>60'


# Create the dataset and split into train and test sets
dataset = TeethXrayDataset(root_dir='/content/gdrive/MyDrive/Teeth X-ray dataset', transform=train_transform)
train_size = int(0.8 * len(dataset))

# Create data loaders for train and test sets
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)



In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import DataLoader


def __getitem__(self, index):
    # Get the filename of the image
    filename = self.file_list[index]

    # Get the age range from the filename
    age_range = get_age_from_filename(filename)

    # Load the image using PIL
    image = Image.open(os.path.join(self.root_dir, filename))

    # Apply the transformation if provided
    if self.transform is not None:
        image = self.transform(image)


    # Map the age range to the corresponding class index
    age_class = age_classes[age_range]

    return image, age_class




# Define the meta-model
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.feature_extractor = torchvision.models.densenet121(pretrained=True).features
        self.conv = nn.Conv2d(1024, 512, kernel_size=3, stride=1, padding=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the hyperparameters
num_classes = 6
learning_rate = 0.001
num_epochs = 10

# Create the dataset and data loader
# Create the dataset and data loader
dataset = TeethXrayDataset(root_dir='/content/gdrive/MyDrive/Teeth X-ray dataset', transform=train_transform)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


# Create an instance of the meta-model

model = ConvNet(num_classes)
model.to(device)
# Define the loss function and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images.unsqueeze(2))  # Reshape the input to add a dummy dimension at channel position
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training loss for every 100 steps
        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}")

print("Training finished!")



NameError: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

# Define your dataset class
class TeethXrayDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # Add code to load and preprocess your dataset here

    def __len__(self):
        # Return the total number of samples in the dataset
        pass

    def __getitem__(self, index):
        # Return a tuple (image, label) for the given index
        pass

# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the hyperparameters
num_classes = 6
learning_rate = 0.001
num_epochs = 10
batch_size = 32

# Create the dataset and data loader
train_transform = transforms.Compose([
    # Add any necessary data transformations here
])
dataset = TeethXrayDataset(root_dir='/content/gdrive/MyDrive/Teeth X-ray dataset', transform=train_transform)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create an instance of the MetaModel
model = MetaModel(num_classes)
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training loss for every 100 steps
        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}")

print("Training finished!")


TypeError: ignored

In [ ]:
import torch
import torchvision.transforms as transforms

class TeethXrayDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        # Load and preprocess your dataset here

    def __len__(self):
        # Return the total number of samples in the dataset
        pass

    def __getitem__(self, index):
        # Return the specific sample at the given index
        pass


In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(224*224*3, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 6)

    def forward(self, x):
        x = x.view(-1, 224*224*3)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# Create data loaders for train sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Train the model
for epoch in range(30):
    running_loss = 0.0
    net.train()
    for images, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print the loss for this epoch
    print('Epoch %d loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))

print('Finished training')



Epoch 1 loss: 0.371
Epoch 2 loss: 0.001
Epoch 3 loss: 0.000
Epoch 4 loss: 0.000
Epoch 5 loss: 0.000
Epoch 6 loss: 0.000
Epoch 7 loss: 0.000
Epoch 8 loss: 0.000
Epoch 9 loss: 0.000
Epoch 10 loss: 0.000
Epoch 11 loss: 0.000
Epoch 12 loss: 0.000
Epoch 13 loss: 0.000
Epoch 14 loss: 0.000
Epoch 15 loss: 0.000
Epoch 16 loss: 0.000
Epoch 17 loss: 0.000
Epoch 18 loss: 0.000
Epoch 19 loss: 0.000
Epoch 20 loss: 0.000
Epoch 21 loss: 0.000
Epoch 22 loss: 0.000
Epoch 23 loss: 0.000
Epoch 24 loss: 0.000
Epoch 25 loss: 0.000
Epoch 26 loss: 0.000
Epoch 27 loss: 0.000
Epoch 28 loss: 0.000
Epoch 29 loss: 0.000
Epoch 30 loss: 0.000
Finished training


In [ ]:
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

# Assuming you have a ToothXRayDataset class that implements the Dataset class

# Split your dataset into training and validation sets
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Create a custom validation dataset class
class ValidationDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Assuming your data consists of image tensors and corresponding labels
        image = self.data[index][0]
        label = self.data[index][1]

        # Preprocess or transform the image if needed

        return image, label

# Create the validation dataset
val_dataset = ValidationDataset(val_data)


# Create data loader for the validation set
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set the model to evaluation mode
net.eval()

# Variables to track the total number of correct predictions and total samples
total_correct = 0
total_samples = 0

# Disable gradient calculation for evaluation
with torch.no_grad():
    for images, labels in val_loader:
        # Forward pass
        outputs = net(images)

        # Get the predicted labels
        _, predicted = torch.max(outputs, 1)

        # Update the counts
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

# Calculate the accuracy
accuracy = total_correct / total_samples * 100

# Print the accuracy
print('Accuracy: %.2f%%' % accuracy)


TypeError: ignored

In [ ]:


# Create the dataset and split into train and test sets
dataset = TeethXrayDataset(root_dir='/content/gdrive/MyDrive/Teeth X-ray dataset', transform=train_transform)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

# Create data loaders for train and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load the DenseNet201 model
model = densenet201(pretrained=True)

# Modify the classifier for age classification
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Conv2d(num_ftrs, len(age_classes), kernel_size=1),
    nn.Flatten(),
    nn.Softmax(dim=1)
)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0

    for labels, images in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")


RuntimeError: ignored

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load the ResNetV2 model
resnet = models.resnet50(pretrained=True)

# Modify the last classification layer
in_features = resnet.fc.in_features
resnet.fc = nn.Sequential(
    nn.Conv2d(in_features, 512, kernel_size=1),  # Replace with a 1x1 convolutional layer
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1)
)

# Add additional fully-connected layers for classification
resnet.fc_age = nn.Sequential(
    nn.Flatten(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, len(age_classes))
)

# Apply softmax activation for age classification
resnet.softmax = nn.Softmax(dim=1)

# Print the modified ResNetV2 model
print(resnet)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 368MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
def __getitem__(self, index):
    # Get the filename of the image
    filename = self.file_list[index]

    # Get the age range from the filename
    age_range = get_age_from_filename(filename)

    # Load the image using PIL
    image = Image.open(os.path.join(self.root_dir, filename))

    # Apply the transformation if provided
    if self.transform is not None:
        image = self.transform(image)

    # Map the age range to the corresponding class index
    age_class = age_classes[age_range]

    # Return the reshaped image tensor and age class tensor
    return image.permute(1, 2, 0), age_class


# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Set the number of epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    # Set the model to training mode
    resnet.train()
    running_loss = 0.0

    for images, labels in train_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = resnet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track the training loss
        running_loss += loss.item() * images.size(0)

    # Calculate the average loss for the epoch
    epoch_loss = running_loss / len(train_dataset)

    # Print the loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')


NameError: ignored

In [ ]:
def __getitem__(self, index):
    # Get the filename of the image
    filename = self.file_list[index]

    # Get the age range from the filename
    age_range = get_age_from_filename(filename)

    # Load the image using PIL
    image = Image.open(os.path.join(self.root_dir, filename))

    # Apply the transformation if provided
    if self.transform is not None:
        image = self.transform(image)

    # Map the age range to the corresponding class index
    age_class = age_classes[age_range]

    # Return the reshaped image tensor and age class tensor
    return image.permute(1, 2, 0), age_class


In [ ]:
class TeethXrayDataset(Dataset):
    # ...

    def __getitem__(self, index):
        # ...
        image = image.unsqueeze(0)  # Add a singleton channel dimension

        # ...


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models


class TeethXrayAgeModel(nn.Module):
    def __init__(self, num_classes):
        super(TeethXrayAgeModel, self).__init__()

        # Load the pre-trained DenseNet201 model
        self.feature_extractor = models.densenet201(pretrained=True)

        # Replace the last classification layer with a 1x1 convolutional layer
        num_features = self.feature_extractor.classifier.in_features
        self.feature_extractor.classifier = nn.Conv2d(num_features, num_classes, kernel_size=1)

        # Add optional attention mechanism here if needed

        # Add fully-connected layers for classification
        self.fc1 = nn.Linear(num_classes, 128)
        self.fc2 = nn.Linear(128, num_classes)

        # Add softmax activation for age classification
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)  # Flatten the feature vector
        features = self.fc1(features)
        features = nn.ReLU()(features)
        features = self.fc2(features)

        # Apply softmax activation for age classification
        age_pred = self.softmax(features)

        return age_pred


# Create an instance of the TeethXrayAgeModel
num_classes = 6  # Number of age classes (modify according to your age classes)
model = TeethXrayAgeModel(num_classes)

# Print the model architecture
print(model)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:01<00:00, 73.4MB/s]


TeethXrayAgeModel(
  (feature_extractor): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

class TeethXrayDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []  # Add code to load and preprocess your dataset here

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Implement code to retrieve an image and its label at the given index
        pass


# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the hyperparameters
num_classes = 6
learning_rate = 0.001
num_epochs = 10
batch_size = 32

# Create the dataset and data loader
train_transform = transforms.Compose([
    # Add any necessary data transformations here
])
dataset = TeethXrayDataset(root_dir='/content/gdrive/MyDrive/Teeth X-ray dataset', transform=train_transform)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

if len(train_dataset) == 0:
    raise ValueError("Empty training dataset. Please check your dataset splitting.")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


# Create an instance of the MetaModel
model = MetaModel(num_classes)
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training loss for every 100 steps
        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}")

print("Training finished!")


ValueError: ignored